Final Project Check-in 2018-11-16
------

Group Name
-----

We are Groot

Student Names
----

1. Jialiang
2. Julia
3. Wei
4. Xiao Han
5. Jessica

Load Data
-----

Fit scikit-learn model
----

Evaluation Metric
----